Анализируем датасэт с информацией об атлетах участвовавших в Олимпийских играх (olymp.csv). Скачайте его. Разделитель запятая.

Данные содержат следующие признаки:
**ID** - уникальный номер спортсмена
**Name** - имя атлета
**Sex** - пол (мужской - M,  женский - F)
**Age** - возраст атлета
**Height** - рост (в см)
**Weight** - вес (в кг)
**Team** - cтрана
**NOC** - код страны от Олимпийского комитета (3 буквы)
**Games** - год и сезон Олимпийских игр
**Year** - год Олимпийских игр
**Season** - сезон (Зимние - Winter,  Лето - Summer)
**City** - город проведения Олимпийских игр
**Sport** - спортивная дисциплина
**Event** - категория спортивной дисциплины
**Medal** - достоинство медали (Золото - Gold, Серебро - Silver, Бронза - Bronze, Без медали - NaN)

## Вопрос 1

Найдите год проведения Олимпиады, в который средний возраст атлетов был минимальный

АЛГОРИТМ РЕШЕНИЯ

Сгруппируйте данные: столбец группировки год/игра (Year/Games), столбец агрегирования возраст (Age), метод агрегирования среднее
Отсортируйте данные по возрастанию с помощью .sort_values()

In [50]:
import pandas as pd

# Загрузим датасет
df = pd.read_csv('olymp.csv', delimiter = ',')

# Группируем данные по годам и вычисляем средний возраст для каждого года
average_age_by_year = df.groupby('Year')['Age'].mean()

# Находим год с минимальным средним возрастом
min_year = average_age_by_year.idxmin()

print(int(min_year))

1896


Год проведения Олимпиады с минимальным средним возрастом атлетов: 1896

## Вопрос 2

Какой вид спорта (Sport) является самым многочисленным с точки зрения количества уникальных атлетов, принявших в нем участие?

In [51]:
# Используем метод nunique() для подсчета уникальных атлетов по виду спорта (Sport)
athlete_count_by_sport = df.groupby('Sport')['ID'].nunique()

# Найдем вид спорта с максимальным количеством уникальных атлетов
most_popular_sport = athlete_count_by_sport.idxmax()

print(most_popular_sport)

Athletics


Самый многочисленный вид спорта по количеству уникальных атлетов: Athletics

## Вопрос 3

Выясните, атлеты из какой страны (Team) завоевали наибольшее количество золотых медалей на Зимней Олимпиаде 1984 года

АЛГОРИТМ РЕШЕНИЯ
Сделайте фильтрацию и оставьте только атлетов завоевавших золотые медали на Зимней Олимпиаде 1984 года
На отфильтрованных данных постройте сводную таблицу: в рядах должны быть названия стран (Team), в колонке ID атлета, в ячейках уникальное количество атлетов из каждой страны. Чтобы получить количество уникальных атлетов воспользуйтесь методом агрегирования .nunique()
Отсортируйте данные по убыванию

In [52]:
# Оставим только атлетов, завоевавших золотые медали на Зимней Олимпиаде 1984 года
winter_1984_gold_medalists = df[(df['Season'] == 'Winter') & (df['Year'] == 1984) & (df['Medal'] == 'Gold')]

# Создадим сводную таблицу, где строки будут странами (Team), а в колонках будут ID атлетов, считаем уникальные значения
unique_athletes_by_team = winter_1984_gold_medalists.pivot_table(index = 'Team', values = 'ID', aggfunc = 'nunique')

# Отсортируем данные по убыванию количества уникальных атлетов
most_gold_medals_country = unique_athletes_by_team.sort_values(by = 'ID', ascending = False).index[0]

print(most_gold_medals_country)

Soviet Union


Страна с наибольшим количеством золотых медалей на Зимней Олимпиаде 1984 года:Soviet Union

## Вопрос 4

Определите минимальное значение разницы между максимальным ростом и весом атлетов из определенной страны. В ответ запишите положительное целое число

АЛГОРИТМ РЕШЕНИЯ

Создайте сводную таблицу: в рядах - короткое название страны (NOC), в колонках - рост (Height) и вес (Weight) атлетов, в ячейках максимальное значение роста и веса по каждой стране
В получившейся сводной таблице создайте новый признак, который будет рассчитан как разность между ростом и весом (по модулю)
По новому признаку найдите минимум

In [53]:
# Создадим сводную таблицу с максимальными значениями роста и веса по каждой стране (NOC)
pivot_table = df.pivot_table(index = 'NOC', values = ['Height', 'Weight'], aggfunc = 'max')

# Создадим новый признак как разницу между максимальным ростом и весом (по модулю)
pivot_table['Height_Weight_Diff'] = abs(pivot_table['Height'] - pivot_table['Weight'])

# Найдем минимальное значение разницы между максимальным ростом и весом
min_difference = pivot_table['Height_Weight_Diff'].min()

print(int(min_difference))

21


Минимальное значение разницы между максимальным ростом и весом атлетов: 21

## Вопрос 5

Определите количество атлетов из Советского Союза (Soviet Union) чей процент успеха на олимпиаде не менее 50%. Показатель процента успеха рассчитывается как частное (количество завоеванных медалей к количеству категорий спортивных дисциплин, в которых они приняли участие) умноженное на 100.

АЛГОРИТМ РЕШЕНИЯ

Сделайте фильтрацию и оставьте только атлетов из Советского союза
По отфильтрованным данным создайте сводную таблицу: в рядах - уникальный номер атлета (ID), в столбцах -  категория спортивной дисциплины (Event) и медали (Medal), в ячейках - количество категорий спортивных дисциплин и количество медалей для каждого атлета. Сохраните сгруппированные данные в новую переменную
В получившемся датафрейме создайте новый признак процента успеха
Сделайте фильтрацию по новому признаку

In [54]:
# Фильтруем данные для атлетов из Советского Союза
soviet_union_df = df[df['Team'] == 'Soviet Union']

# Создаем сводную таблицу, где строки - уникальные номера атлетов (ID), столбцы - категории спортивных дисциплин (Event) и медали (Medal), а значения - количество категорий спортивных дисциплин и количество медалей для каждого атлета
pivot_table = soviet_union_df.pivot_table(index = 'ID', columns = ['Event', 'Medal'], values = 'Year',
										  aggfunc = 'count', fill_value = 0)

# Создаем новый признак "Процент успеха" для каждого атлета
pivot_table['Success_Rate'] = (pivot_table.sum(axis = 1) / pivot_table.count(axis = 1)) * 100

# Фильтруем атлетов, чей процент успеха не менее 50%
athletes_with_success_rate_above_50 = pivot_table[pivot_table['Success_Rate'] >= 50]

# Подсчитываем количество таких атлетов
count_athletes_above_50_percent = len(athletes_with_success_rate_above_50)

print(count_athletes_above_50_percent)


0


Количество атлетов из Советского Союза с процентом успеха не менее 50%: 0

## Вопрос 6

Определите количество Олимпиад, на которых доля женщин больше, чем средняя доля их участия в целом

АЛГОРИТМ РЕШЕНИЯ

Создайте сводную таблицу: в рядах - олимпиада (Games), в колонке - уникальный номер атлета (ID), в ячейках  - количество уникальных атлетов в каждому году. Сохраните сводную таблицу в переменную
Создайте сводную таблицу только по женщинам: в рядах - олимпиада (Games), в колонке - уникальный номер атлетки (ID), в ячейках  - количество уникальных атлеток в каждому году. Сохраните сводную таблицу в переменную
Найдите долю женщин в каждому году (найдите частное сводных таблиц). Получившуюся таблицу сохраните в новую переменную
Сделайте фильтрацию и оставьте, только те Олимпиады, где доля женщин больше чем среднее значение доли их участия
Посчитайте количество таких Олимпиад с метода .count()

In [55]:
# Создаем сводную таблицу, где строки - олимпиада (Games), столбцы - уникальные номера атлетов (ID), значения - количество уникальных атлетов для каждой олимпиады
athlete_count_by_year = df.pivot_table(index = 'Games', values = 'ID', aggfunc = 'nunique')

# Создаем сводную таблицу только для женщин, где строки - олимпиада (Games), столбцы - уникальные номера атлеток (ID), значения - количество уникальных атлеток для каждой олимпиады
female_athlete_count_by_year = df[df['Sex'] == 'F'].pivot_table(index = 'Games', values = 'ID', aggfunc = 'nunique')

# Вычисляем долю женщин в каждой олимпиаде
female_percentage_by_year = female_athlete_count_by_year / athlete_count_by_year

# Находим среднюю долю женщин по всем олимпиадам
mean_female_percentage = female_percentage_by_year.mean()

# Фильтруем олимпиады, где доля женщин больше средней доли
olympics_with_above_average_female_percentage = female_percentage_by_year[
	female_percentage_by_year > mean_female_percentage]

# Подсчитываем количество таких олимпиад
count_olympics_with_above_average_female_percentage = olympics_with_above_average_female_percentage.count()

print(count_olympics_with_above_average_female_percentage)

ID    24
dtype: int64


Количество Олимпиад, на которых доля женщин больше средней доли их участия: 

## Вопрос 7

Создайте функцию medal_weight, которая на основании столбца Medal, определяет количество очков за медали разного достоинства:

3 - Gold
2 - Silver
1 - Bronze
0 - NaN

In [56]:
# Определение функции medal_weight
def medal_weight(medal):
	if medal == 'Gold':
		return 3
	elif medal == 'Silver':
		return 2
	elif medal == 'Bronze':
		return 1
	else:
		return 0


# Применение функции к столбцу "Medal" и создание нового столбца "MedalWeight"
df['MedalWeight'] = df['Medal'].apply(medal_weight)

# Вывод первых нескольких строк для проверки
print(df[['Medal', 'MedalWeight']].head())

  Medal  MedalWeight
0   NaN            0
1   NaN            0
2   NaN            0
3  Gold            3
4   NaN            0


```python
# Определение функции medal_weight
def medal_weight(medal):
    if medal == 'Gold':
        return 3
    elif medal == 'Silver':
        return 2
    elif medal == 'Bronze':
        return 1
    else:
        return 0

# Применение функции к столбцу "Medal" и создание нового столбца "MedalWeight"
df['MedalWeight'] = df['Medal'].apply(medal_weight)

# Вывод первых нескольких строк для проверки
print(df[['Medal', 'MedalWeight']].head())
```

## Вопрос 8

Датафрейм лежит в переменной olmp. Примените функцию созданную в прошлом задании к столбцу Medal и создайте новый признак Medal_score в датафрейме olmp.

In [57]:
df['Medal_score'] = df['Medal'].apply(medal_weight)

```python
# Применение функции к столбцу "Medal" и создание нового признака "Medal_score"
olmp['Medal_score'] = olmp['Medal'].apply(medal_weight)
```

## Вопрос 9

Определите тип данных с точки зрения анализа в новом столбце Medal_score

Вопрос 9

- категориальный порядковый
- числовой дискретный
- категориальный номинальный бинарный
- категориальный номинальный
- числовой непрерывный

In [58]:
# Определяем тип данных в столбце 'Medal_score'
data_type_medal_score = df['Medal_score'].dtype

# Выводим тип данных
print("Тип данных в столбце 'Medal_score':", data_type_medal_score)

Тип данных в столбце 'Medal_score': int64


Тип данных в новом столбце "Medal_score" является числовым дискретным. Этот столбец содержит целые числа, представляющие количество очков за медали разного достоинства (3, 2, 1, 0).

## Вопрос 10

Узнайте имя атлета (Name), который получил суммарно больше всего очков по новому столбцу Medal_score

In [59]:
# Группируем данные по имени атлета и суммируем их Medal_score
athlete_scores = df.groupby('Name')['Medal_score'].sum()

# Находим имя атлета с наибольшей суммой очков
top_athlete = athlete_scores.idxmax()

print(top_athlete)

Michael Fred Phelps, II


Имя атлета с наибольшей суммой очков: Michael Fred Phelps, II